In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from itertools import combinations
from scipy import stats
import random
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier,RandomForestClassifier,StackingClassifier,VotingClassifier
from xgboost import XGBClassifier
 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm

In [3]:
import seaborn as sns
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn import preprocessing 

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
train_df = pd.read_csv("train_ctrUa4K.csv")
#check the info of the dataset
print(train_df.shape)
train_df['Dependents'] = train_df['Dependents'].replace(['3+'],'3')
train_df['Married']=train_df['Married'].fillna('No')
train_df.info()
print('-'*30)
# resource_data = pd.read_csv("/content/drive/My Drive/ Deep learning assignments/ LSTM Donors_choose/resources.csv")
# #check the info of the dataset
# resource_data .info()

(614, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
------------------------------


In [6]:
train_df.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [7]:
test_df= pd.read_csv("test_lAUu6dG.csv")
test_df['Dependents'] = test_df['Dependents'].replace(['3+'],'3')
test_df['Loan_Amount_Term'] = test_df['Loan_Amount_Term'].replace([6.0],12.0)
test_df['Loan_Amount_Term'] = test_df['Loan_Amount_Term'].replace([350.0],360.0)
#check the info of the dataset
print(test_df.shape)
# test_df.fillna(-1,inplace=True)
test_df.info()

(367, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [8]:
nan_=['Self_Employed','Credit_History','Loan_Amount_Term','Gender','Dependents','LoanAmount']

def indecies(df,l):
    nan_col_indecies=[];nan_indecies=[]
    for i in nan_:
        s=df[df[i].isnull()].index.tolist()
        nan_col_indecies.append(s)
        nan_indecies.extend(s)
    return nan_col_indecies,nan_indecies

In [9]:
nan_col_indecies_tr,nan_indecies_tr = indecies(train_df,nan_)
nan_col_indecies_te,nan_indecies_te = indecies(test_df,nan_)

In [10]:
sample_df=pd.read_csv('sample_submission_49d68Cx.csv')
sample_df.head(10)

,Loan_ID,Loan_Status
0,LP001015,N
1,LP001022,N
2,LP001031,N
3,LP001035,N
4,LP001051,N
5,LP001054,N
6,LP001055,N
7,LP001056,N
8,LP001059,N
9,LP001067,N


In [11]:
num_columns=['ApplicantIncome','CoapplicantIncome','LoanAmount']
cat_columns=['Gender','Married','Dependents','Education','Self_Employed','Loan_Amount_Term','Credit_History','Property_Area',]#'Loan_Status']

In [12]:
#we an use any method to impute
#fill null values with mode
# for i in nan_:
#     train_df[i]=train_df[i].fillna(train_df[i].mode()[0])
#     test_df[i]=test_df[i].fillna(train_df[i].mode()[0])

In [13]:
le=LabelEncoder()
train_df[cat_columns] = train_df[cat_columns].apply(le.fit_transform)

test_df[cat_columns] = test_df[cat_columns].apply(le.fit_transform)

In [14]:
output_labels=LabelEncoder()
train_df['Loan_Status']=output_labels.fit_transform(train_df['Loan_Status'])

In [15]:
t1=train_df.copy()
t2=test_df.copy()
 

#t1.drop(t1.index[nan_indecies_tr[4]])
t1.drop(set(nan_indecies_tr),axis=0,inplace=True)
t2.drop(set(nan_indecies_te),axis=0,inplace=True)
t_tr=pd.concat([t1,t2])
t_tr.shape

(769, 13)

In [16]:
col_1=['Married','Education','ApplicantIncome','CoapplicantIncome','Property_Area']
trp=[];tep=[]
for i in range(len(nan_)):
    X=t_tr[col_1]
    y=t_tr[nan_[i]]
    model_= CatBoostClassifier()
    model_.fit(X,y)
    p1=[];p2=[]
    for j in range(len(nan_col_indecies_tr[i])):
        p=model_.predict(train_df[X.columns][nan_col_indecies_tr[i][j]:nan_col_indecies_tr[i][j]+1])
        train_df[nan_[i]][nan_col_indecies_tr[i][j]:nan_col_indecies_tr[i][j]+1]=p[0]
        p1.append(p)
    trp.append(p1)
    for j in range(len(nan_col_indecies_te[i])):
        p=model_.predict(test_df[X.columns][nan_col_indecies_te[i][j]:nan_col_indecies_te[i][j]+1])
        test_df[nan_[i]][nan_col_indecies_te[i][j]:nan_col_indecies_te[i][j]+1]=p[0]
        p2.append(p)
    tep.append(p2)
    col_1.append(str(nan_[i]))
    print('-'*10)

Learning rate set to 0.009209
0:	learn: 0.6851324	total: 100ms	remaining: 1m 40s
1:	learn: 0.6770957	total: 103ms	remaining: 51.2s
2:	learn: 0.6706298	total: 105ms	remaining: 34.8s
3:	learn: 0.6628949	total: 107ms	remaining: 26.6s
4:	learn: 0.6568245	total: 108ms	remaining: 21.5s
5:	learn: 0.6490271	total: 111ms	remaining: 18.3s
6:	learn: 0.6414507	total: 113ms	remaining: 16.1s
7:	learn: 0.6356315	total: 116ms	remaining: 14.3s
8:	learn: 0.6298564	total: 118ms	remaining: 13s
9:	learn: 0.6231425	total: 120ms	remaining: 11.9s
10:	learn: 0.6172293	total: 122ms	remaining: 10.9s
11:	learn: 0.6110048	total: 123ms	remaining: 10.1s
12:	learn: 0.6060124	total: 124ms	remaining: 9.44s
13:	learn: 0.6000625	total: 126ms	remaining: 8.91s
14:	learn: 0.5952408	total: 128ms	remaining: 8.38s
15:	learn: 0.5898824	total: 130ms	remaining: 7.97s
16:	learn: 0.5838610	total: 132ms	remaining: 7.61s
17:	learn: 0.5779895	total: 134ms	remaining: 7.29s
18:	learn: 0.5736023	total: 135ms	remaining: 6.98s
19:	learn: 0

In [17]:
# train_df.to_csv('train_1.csv',index=False)
# test_df.to_csv('test_1.csv',index=False)

In [18]:
train_df.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,141.0,8,1,2,1
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,8,1,0,0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,8,1,2,1
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,8,1,2,1
4,LP001008,1,0,0,0,0,6000,0.0,141.0,8,1,2,1
5,LP001011,1,1,2,0,1,5417,4196.0,267.0,8,1,2,1
6,LP001013,1,1,0,1,0,2333,1516.0,95.0,8,1,2,1
7,LP001014,1,1,3,0,0,3036,2504.0,158.0,8,0,1,0
8,LP001018,1,1,2,0,0,4006,1526.0,168.0,8,1,2,1
9,LP001020,1,1,1,0,0,12841,10968.0,349.0,8,1,1,0


In [19]:
#combine two categorical columns
comb=combinations(cat_columns,2)

combined_cols2=[]
for i in comb:
    train_df[i[0]+'_'+i[1]]= train_df[i[0]].astype(str)+'_'+train_df[i[1]].astype(str)
    test_df[i[0]+'_'+i[1]]= test_df[i[0]].astype(str)+'_'+test_df[i[1]].astype(str)
    combined_cols2.append(i[0]+'_'+i[1])

In [20]:
#combined columns 
n=13
for i in train_df.columns[13:41]:
    print(i,n)
    n+=1

Gender_Married 13
Gender_Dependents 14
Gender_Education 15
Gender_Self_Employed 16
Gender_Loan_Amount_Term 17
Gender_Credit_History 18
Gender_Property_Area 19
Married_Dependents 20
Married_Education 21
Married_Self_Employed 22
Married_Loan_Amount_Term 23
Married_Credit_History 24
Married_Property_Area 25
Dependents_Education 26
Dependents_Self_Employed 27
Dependents_Loan_Amount_Term 28
Dependents_Credit_History 29
Dependents_Property_Area 30
Education_Self_Employed 31
Education_Loan_Amount_Term 32
Education_Credit_History 33
Education_Property_Area 34
Self_Employed_Loan_Amount_Term 35
Self_Employed_Credit_History 36
Self_Employed_Property_Area 37
Loan_Amount_Term_Credit_History 38
Loan_Amount_Term_Property_Area 39
Credit_History_Property_Area 40


In [21]:
#print feature elements present only in test and their column names
n=13
for i in train_df.keys()[13:41]:
    flag=False
    l1=train_df[i].value_counts().keys();l2=test_df[i].value_counts().keys()
    for v in l2:
        if v not in(l1):
            flag=True
            print(v)
    if flag:
        print(i)
        print('-'*20)
    n+=1

0_3
0_0
1_1
Married_Loan_Amount_Term
--------------------
0_3
1_2
3_6
3_4
2_0
Dependents_Loan_Amount_Term
--------------------
1_0
Self_Employed_Loan_Amount_Term
--------------------
0_0
Loan_Amount_Term_Credit_History
--------------------
6_0
0_0
0_1
Loan_Amount_Term_Property_Area
--------------------


In [22]:
train_df['Married_Loan_Amount_Term'].value_counts(),test_df['Married_Loan_Amount_Term'].value_counts()

(1_8    336
 0_8    188
 1_5     37
 0_9     10
 1_7     10
 0_5      9
 1_9      5
 1_3      4
 0_7      3
 0_6      2
 0_1      2
 1_6      2
 1_4      2
 1_2      1
 0_4      1
 0_2      1
 1_0      1
 Name: Married_Loan_Amount_Term, dtype: int64,
 1_8    202
 0_8    116
 1_5     13
 0_5      9
 1_9      7
 0_7      4
 1_6      3
 1_7      3
 0_3      2
 0_0      1
 0_9      1
 1_4      1
 1_1      1
 1_2      1
 0_6      1
 1_0      1
 1_3      1
 Name: Married_Loan_Amount_Term, dtype: int64)

In Married_Loan_Amount_Term column <br />
0_3 is present only in the test and has the value count of 2.There is no element in test set with that value we can select 
element with least count substitute(ex 0_0).<br />

similarly 1_1 is present only in test and has value of 1.And elements such as 1_2,0_6,0_9,1_0,1_3,1_4 also have value of 1
teir corresponding values in test set are 1,2,10,1,1,1 mode of this is 1 and 1_2,1_0,1_3,1_4 all have value of 1 we can randomly 
select any element and substitute.Reason for substituting randomly from thesevalues is to not to increase value count of one 
particular element.

In [23]:
train_df['Dependents_Loan_Amount_Term'].value_counts(),test_df['Dependents_Loan_Amount_Term'].value_counts()

(0_8    313
 2_8     88
 1_8     83
 3_8     40
 0_5     21
 0_9     11
 1_5     11
 3_5      8
 2_5      6
 0_7      6
 2_7      3
 1_6      2
 3_7      2
 2_3      2
 0_4      2
 1_3      2
 1_7      2
 2_9      2
 3_9      1
 0_1      1
 1_9      1
 2_6      1
 3_2      1
 0_2      1
 0_0      1
 1_1      1
 2_4      1
 0_6      1
 Name: Dependents_Loan_Amount_Term, dtype: int64,
 0_8    181
 2_8     52
 1_8     52
 3_8     33
 0_5     12
 0_7      6
 2_5      5
 3_5      5
 0_9      4
 2_9      2
 1_6      2
 0_3      2
 1_2      1
 2_6      1
 1_7      1
 1_9      1
 3_9      1
 3_6      1
 0_1      1
 3_4      1
 0_0      1
 2_0      1
 1_3      1
 Name: Dependents_Loan_Amount_Term, dtype: int64)

In Dependents_Loan_Amount_Term column <br />
0_3 is present only in the test and has the value count of 2.In the test set two other elements 1_6 and 2_9 also have the same value 
of 2.To substitute the proper value for 0_3 we can choose either take value of 1_6 or 2_9 and both have same value count of 2 so any 
of them can be substituted.<br />

smilarly for 1_2 which has value of 1 and in test set 1_9,3_9,2_6,1_7,1_3,0_0,0_1 also have value of 1 and their corresponding values 
in test set are 1,1,1,1,1,1,1 we can randomly select any one of them and substitute 

In [ ]:
#instaed of replacing individual element we can write a function
test_df['Married_Loan_Amount_Term'] = test_df['Married_Loan_Amount_Term'].replace(['0_3'],'1_0')
test_df['Married_Loan_Amount_Term'] = test_df['Married_Loan_Amount_Term'].replace(['1_1'],'0_6')
test_df['Married_Loan_Amount_Term'] = test_df['Married_Loan_Amount_Term'].replace(['0_0'],'0_6')

test_df['Dependents_Loan_Amount_Term'] = test_df['Dependents_Loan_Amount_Term'].replace(['0_3'],'1_6')
test_df['Dependents_Loan_Amount_Term'] = test_df['Dependents_Loan_Amount_Term'].replace(['1_2'],'3_9')
test_df['Dependents_Loan_Amount_Term'] = test_df['Dependents_Loan_Amount_Term'].replace(['2_0'],'2_6')
test_df['Dependents_Loan_Amount_Term'] = test_df['Dependents_Loan_Amount_Term'].replace(['3_4'],'1_3')
test_df['Dependents_Loan_Amount_Term'] = test_df['Dependents_Loan_Amount_Term'].replace(['3_6'],'0_0')

test_df['Self_Employed_Loan_Amount_Term'] = test_df['Self_Employed_Loan_Amount_Term'].replace(['1_0'],'0_1')

test_df['Loan_Amount_Term_Credit_History'] = test_df['Loan_Amount_Term_Credit_History'].replace(['0_0'],'1_1')

test_df['Loan_Amount_Term_Property_Area'] = test_df['Loan_Amount_Term_Property_Area'].replace(['0_0'],'1_1')
test_df['Loan_Amount_Term_Property_Area'] = test_df['Loan_Amount_Term_Property_Area'].replace(['0_1'],'1_1')
test_df['Loan_Amount_Term_Property_Area'] = test_df['Loan_Amount_Term_Property_Area'].replace(['6_0'],'7_2')

In [24]:
#it returns similar keys 
def similar_(k,col,missing):
    missing=missing
    d_tr=train_df[col].value_counts() #create dictionary of value counts
    d_te=test_df[col].value_counts()
    keys=[];tr_values=[]
    for i in d_te.keys():#traverse through test keys 
        if d_te[i]==d_te[k]:#if there is similar key
            if i not in missing:#and if it is not present in missing list
                    keys.append(i)#append that key
                    tr_values.append(d_tr[i])#and its corresponding test value
    return keys,tr_values,d_tr,d_te

In [25]:
def replace_(missing,col): 
    keys=[]
    for i in missing:
        k,v,d_tr,d_te=similar_(i,col,missing)
        s_keys=[]
        for i in k:
            if d_tr[i]==stats.mode(v)[0][0]:
                s_keys.append(i)
        if len(s_keys)==0:#if there is no similar key for missing element
            min=10;u_key=0 #find the one with with smallest value 
            for i in d_te.keys():
                if d_te[i]<=min and i not in missing:
                    min=d_te[i]
                    u_key=i
            keys.append(u_key)
        else:
            keys.append(random.choice(s_keys))#if there are similar keys select one randomly 
    return keys

In [26]:
#this code snippet from replace_ function returns missing values
missing=[]
l1=train_df['Dependents_Loan_Amount_Term'].value_counts().keys();l2=test_df['Dependents_Loan_Amount_Term'].value_counts().keys()
for v in l2:
    if v not in(l1):
        missing.append(v)
missing

['0_3', '1_2', '3_6', '3_4', '2_0']

In [28]:
# for replace_ function we pass missing values and column name that has missing values
# then the similar function takes each missing value, column name and missing list as input
k,v,d_tr,d_te=similar_(missing[0],'Dependents_Loan_Amount_Term',missing)

#it returns similar keys their values and value counts of train and test data
k,v,d_tr,d_te

(['2_9', '1_6'],
 [2, 2],
 0_8    313
 2_8     88
 1_8     83
 3_8     40
 0_5     21
 0_9     11
 1_5     11
 3_5      8
 2_5      6
 0_7      6
 2_7      3
 1_6      2
 3_7      2
 2_3      2
 0_4      2
 1_3      2
 1_7      2
 2_9      2
 3_9      1
 0_1      1
 1_9      1
 2_6      1
 3_2      1
 0_2      1
 0_0      1
 1_1      1
 2_4      1
 0_6      1
 Name: Dependents_Loan_Amount_Term, dtype: int64,
 0_8    181
 2_8     52
 1_8     52
 3_8     33
 0_5     12
 0_7      6
 2_5      5
 3_5      5
 0_9      4
 2_9      2
 1_6      2
 0_3      2
 1_2      1
 2_6      1
 1_7      1
 1_9      1
 3_9      1
 3_6      1
 0_1      1
 3_4      1
 0_0      1
 2_0      1
 1_3      1
 Name: Dependents_Loan_Amount_Term, dtype: int64)

In [29]:
#this code replaces missing values
for i in train_df.keys()[13:41]:
    missing=[]
    l1=train_df[i].value_counts().keys();l2=test_df[i].value_counts().keys()
    for k in l2:
        if k not in(l1):
            missing.append(k)
    if len(missing)!=0:
        keys=replace_(missing,i)
        for r in range(len(missing)):
            test_df[i] = test_df[i].replace(missing[r],keys[r])

In [30]:
def frequency_encoding(column_name,output_column_name,df1,df2):
    fe_pol = (df1.groupby(column_name).size()) / len(df1)
    df2[output_column_name] = df2[column_name].apply(lambda x : fe_pol[x])

In [31]:
for i in cat_columns:
    frequency_encoding(i,i+'_Ren',train_df,train_df)
    frequency_encoding(i,i+'_Ren',train_df,test_df)
    print(i)

Gender
Married
Dependents
Education
Self_Employed
Loan_Amount_Term
Credit_History
Property_Area


In [32]:
for i in combined_cols2:
    print(i)
    frequency_encoding(i,i+'_Ren',train_df,train_df)
    frequency_encoding(i,i+'_Ren',train_df,test_df)

Gender_Married
Gender_Dependents
Gender_Education
Gender_Self_Employed
Gender_Loan_Amount_Term
Gender_Credit_History
Gender_Property_Area
Married_Dependents
Married_Education
Married_Self_Employed
Married_Loan_Amount_Term
Married_Credit_History
Married_Property_Area
Dependents_Education
Dependents_Self_Employed
Dependents_Loan_Amount_Term
Dependents_Credit_History
Dependents_Property_Area
Education_Self_Employed
Education_Loan_Amount_Term
Education_Credit_History
Education_Property_Area
Self_Employed_Loan_Amount_Term
Self_Employed_Credit_History
Self_Employed_Property_Area
Loan_Amount_Term_Credit_History
Loan_Amount_Term_Property_Area
Credit_History_Property_Area
